In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from tqdm import tqdm, trange

In [ ]:
tf.reset_default_graph()
# Insights
# It's accepted that MLP could do the classification all right.

# Prepare the data for ML algorithms
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# Select a model
l = mnist.train.num_examples
m = 28 * 28
n = 10
activation_type = 'relu'
hidden_layers_shape = [200, 100] # 200,000 variables for 40000 samples. I see. # shorthand
learning_rate = 0.01
n_epochs = 100
minibatch_size = 128
n_minibatches_per_epoch = mnist.train.num_examples / minibatch_size

# Graph
X = tf.placeholder(name="X", shape=[None,m], dtype=tf.float32)
y0 = tf.placeholder(name="y0", shape=[None], dtype=tf.int32)
hidden1 = tf.layers.dense(X, units=hidden_layers_shape[0], activation=tf.nn.relu, name="hidden1")
hidden2 = tf.layers.dense(hidden1, units=hidden_layers_shape[1], activation=tf.nn.relu, name="hidden2")
y = tf.layers.dense(hidden2, n, activation=tf.nn.relu, name="y")

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y0, logits=y), name="loss")
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
summary_op = tf.summary.scalar("loss", loss)

# Session
sess = tf.Session()
sess.run(tf.global_variables_initializer())
filewriter = tf.summary.FileWriter('logs/tf.' + time.strftime("%Y%m%d%H%M%S"),
                                  tf.get_default_graph())
for epoch in trange(n_epochs):
    for minibatch in range(mnist.train.num_examples / minibatch_size):
        X_batch, y0_batch = mnist.train.next_batch(minibatch_size)
        _summary_op, _ = sess.run([summary_op, train_op], feed_dict={X: X_batch, y0: y0_batch})
        filewriter.add_summary(_summary_op, epoch * n_minibatches_per_epoch + minibatch)
filewriter.close()
sess.close()
        

In [9]:
# basic RNN
tf.reset_default_graph()

# get the data
_X0 = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]
_X1 = [[9, 8, 7], [6, 5, 4], [3, 2, 1], [0, 9, 8]]
_Y0 = [[1, 2], [3, 4], [5, 6], [7, 8]]
_Y1 = [[9, 8], [7, 6], [5, 4], [3, 2]]

# select the model
n_timesteps = 2
l = 4
m = 3
n = 2
hidden_layers_shape = 5

# graph
X0 = tf.placeholder(shape=[None, m], name="X0", dtype=tf.float32)
X1 = tf.placeholder(shape=[None, m], name="X1", dtype=tf.float32)

Wxy = tf.get_variable("Wxy", shape=[m, n], dtype=tf.float32)
Wyy = tf.get_variable("Wyy", shape=[n, n], dtype=tf.float32)
Y0hat = tf.nn.elu(tf.matmul(X0, Wxy))
Y1hat = tf.nn.elu(tf.matmul(X1, Wxy) + tf.matmul(Y0hat, Wyy))

# session
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run([Y0hat, Y1hat], feed_dict={X0: _X0, X1: _X1})

[array([[ 2.5672605 , -0.39596552],
        [ 7.089168  , -0.936262  ],
        [11.611074  , -0.99327433],
        [-0.23969859, -0.99860835]], dtype=float32),
 array([[13.84283   , -0.9908481 ],
        [14.219775  ,  0.09620595],
        [14.254899  ,  4.3124247 ],
        [14.30664   ,  0.9156122 ]], dtype=float32)]

In [25]:
# jump to tensorflow.contrib's BasicRNNCell, a blackbox
tf.reset_default_graph()

n_instances = 3
n_timesteps = 2
n_features = 1
n_labels = 1
_X = [[[1],[2]], [[3],[4]], [[5],[0]]]
_y = [[[1],[2]], [[3],[4]], [[5],[0]]]
sequence_length = [2, 2, 1]

X = tf.placeholder(shape=[None, n_timesteps, n_features], dtype=tf.float32, name="X")
y = tf.placeholder(shape=[None, n_labels], dtype=tf.float32, name="y")
yhats, yhat = tf.nn.dynamic_rnn(tf.contrib.rnn.BasicRNNCell(n_labels), X, dtype=tf.float32, sequence_length=sequence_length)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
_yhats, _yhat = sess.run([yhats, yhat], feed_dict={X: _X})

In [ ]:
# Using the tf.contrib.rnn.BasicRNNCell blackbox for training and classifying MNIST
tf.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data')
_X_test = mnist.test.images.reshape((-1, 28, 28))
_y_test = mnist.test.labels

n_instances = mnist.train.num_examples
n_timesteps = 28
n_features = 28
n_output = 10
n_labels = 1
hidden_RNNlayer_shape = 150
learning_rate = 0.001
n_epochs = 100
minibatch_size = 150
n_minibatches_per_epoch = n_instances / minibatch_size

X = tf.placeholder(name="X", shape=[None, n_timesteps, n_features], dtype=tf.float32)
y = tf.placeholder(name="y", shape=[None], dtype=tf.int32)
hidden_MultiRNN = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicRNNCell(hidden_RNNlayer_shape) for _ in range(2)])
hs, h = tf.nn.dynamic_rnn(hidden_MultiRNN, X, dtype=tf.float32)
yhat = tf.layers.dense(h[-1], units=n_output, activation=tf.nn.relu, name="yhat")
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=yhat))
training_op = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True).minimize(loss)
summary_op = tf.summary.scalar("loss", loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
filewriter = tf.summary.FileWriter('logs/tf.' + time.strftime("%Y%m%d%H%M%S"), 
                                  tf.get_default_graph())
for epoch in trange(n_epochs):
    for minibatch in range(n_minibatches_per_epoch):
        _X_batch, _y_batch = mnist.train.next_batch(minibatch_size)
        _X_batch = _X_batch.reshape((-1, n_timesteps, n_features))
        _summary_op, _ = sess.run([summary_op, training_op], feed_dict={X: _X_batch, y: _y_batch})
        filewriter.add_summary(_summary_op, epoch * n_minibatches_per_epoch + minibatch)